<a href="https://colab.research.google.com/github/KirtiNayak11/NLP/blob/main/english_to_hindi_translator_machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/hindi-english-truncated-corpus/Hindi_English_Truncated_Corpus.csv')

In [ ]:
df.head()

In [ ]:
df['source'].value_counts()

In [ ]:
df_copy = df.copy()

In [ ]:
df = df[df['source']=='ted']

In [ ]:
df.head()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df = df.sample(n=25000,random_state=101)
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

**Data Preprocessing**

In [ ]:
df['english_sentence'] = df['english_sentence'].apply(lambda x:x.lower())
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x:x.lower())

In [ ]:
import re

In [ ]:
df['english_sentence'] = df['english_sentence'].apply(lambda x : re.sub("'",'',x))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x : re.sub("'",'',x))

In [ ]:
import string
from string import digits

In [ ]:
exclude = set(string.punctuation)
df['english_sentence'] = df['english_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))

In [ ]:
remove_digits = str.maketrans('','',digits)
df['english_sentence']=df['english_sentence'].apply(lambda x:x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:re.sub("[२३०८१५७९४६]","",x))

In [ ]:
df['english_sentence']=df['english_sentence'].apply(lambda x:x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x:re.sub(" +"," ",x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:re.sub(" +"," ",x))

In [ ]:
df['english_sentence'] = df['english_sentence'].apply(lambda x:re.sub("a-zA-Z"," ",x))

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
corpus=' '
for line in df['english_sentence']:
    corpus+=' '.join(line.split())
corpus

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(stopwords=stop_words,background_color='white',colormap='Dark2',max_font_size=150)
wc.generate(corpus)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (16,6)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

plt.show()

In [ ]:
input_texts=[]
target_texts=[]

input_characters=set()
target_characters=set()

In [ ]:
for line in df['english_sentence']:
    input_texts.append(line)
    for char in line:
        if re.findall('[a-zA-Z]',char) or char==' ':
            if char not in input_characters:
                input_characters.add(char)

for line in df['hindi_sentence']:
    target_texts.append('%' + line + '$')

    for char in target_texts:
        if char not in target_characters:
            target_characters.add(char)


In [ ]:
input_characters = sorted(list(input_characters))
output_charactes = sorted(list(output_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in output_texts])

In [ ]:
print("Number of samples : ", len(input_texts))
print("Number of unique input tokens : ", num_encoder_tokens)
print("Number of unique output tokens : ", num_decoder_tokens)
print("Max sequence length for inputs : ", max_encoder_seq_length)
print("Max sequence length for outputs : ", max_decoder_seq_length)

In [ ]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

**Encoder and Decoder Architecture**

In [ ]:
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(target_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [ ]:
print("Encoder input data : ", encoder_input_data.shape)
print("Decoder input data : ", decoder_input_data.shape)
print("Decoder target data : ", decoder_target_data.shape)

# Coverting text into one hot encoding

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t,char in enumerate(input_text):
        if re.findall("[a-zA-Z]",char) or char==' ':
            encoder_input_data[i,t,input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[' ']]=1

    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1

        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1

    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [ ]:
batch_size = 128 # Batch size for training
epochs = 50 # Number of epochs to train for
latent_dim = 256

**Encoder and Decoder Architecture**

In [ ]:
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(target_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [ ]:
print("Encoder input data : ", encoder_input_data.shape)
print("Decoder input data : ", decoder_input_data.shape)
print("Decoder target data : ", decoder_target_data.shape)

**Encoder LSTM model**

In [ ]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)

encoder_states = [state_h,state_c]

In [ ]:
decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_ =decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [ ]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Encoder LSTM model

In [ ]:
checkpoint_path = "/kaggle/working/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
         callbacks=[cp_callback])

In [ ]:
encoder_model = Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_state_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_state_inputs)

decoder_states=[state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model =Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    state_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1,num_decoder_tokens))

    target_seq[0, 0, target_token_index['%']] = 1

    stop_condition = False

    decoded_sentence = ''

    while not stop_condition:

        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sample_char = reverse_target_char_index(sampled_token_index)

        decoded_sentence+=sampled_char

    if (sampled_char=='$' or
       len(decoded_sentence)>max_decoder_seq_length):
         stop_condition = True

    target_seq = np.zeros((1,1,num_encoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    states_value = [h, c]


    return decoded_sentence


In [ ]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence : ', input_texts[seq_index])
    print('Decoded sentence : ', decoded_sentence)